In [1]:
import os
import sys
import pandas as pd
import argparse
from datetime import datetime, timedelta
import pathlib

# 新增：將上一層目錄加入 sys.path
parent_path = pathlib.Path().absolute().parent
sys.path.insert(0, str(parent_path))
# Setup Django environment
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'website_configs.settings')
import django
django.setup()
# 重要：設定環境變數以允許在 Jupyter 的異步環境中執行同步操作
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Now we can import Django models
from app_user_keyword_db.models import NewsData

In [2]:
from django.db.models import Q, Max, F
from collections import Counter

In [5]:
# Searching keywords from "content" column
# This function now uses database queries instead of pandas
def filter_database_fullText(user_keywords, cond, cate, weeks):
    # Get the latest date in the database
    latest_date = NewsData.objects.aggregate(max_date=Max('date'))['max_date']
    
    # Calculate start date
    start_date = latest_date - timedelta(weeks=weeks)
    
    # Base query - filter by date range
    queryset = NewsData.objects.filter(date__gte=start_date, date__lte=latest_date)
    
    # Filter by category if not "全部"
    if cate != "全部":
        queryset = queryset.filter(category=cate)
    
    # Filter by keywords based on condition (AND or OR)
    if cond == 'and':
        # For AND condition, we need all keywords to be present
        for kw in user_keywords:
            queryset = queryset.filter(content__contains=kw)
    elif cond == 'or':
        # For OR condition, any keyword can be present
        q_objects = Q()
        for kw in user_keywords:
            q_objects |= Q(content__contains=kw)
        queryset = queryset.filter(q_objects)
    
    return queryset

In [6]:

user_keywords = ['烏克蘭', '俄羅斯']  # Example keywords
cond = 'and'  # Example condition (and/or), 
cate = '全部'  # Example category (or "全部" for all categories)
weeks = 4  # Example weeks
queryset = filter_database_fullText(user_keywords, cond, cate, weeks)

In [7]:
for news in queryset:
    print(news.title)

對普亭很生氣 川普嗆關稅升級鎖定石油
雙普通話推進停火 各有盤算變數眾多
雙普通話　同意俄烏能源設施停火30天
烏克蘭恐遭宰割？ 川普與普亭今晚通話商討俄烏停火
澤倫川普重新修好 美烏續談俄烏停火
美特使：川普擬本週與普亭通話　商討烏俄戰爭停火方...
原則上同意停火30天　普亭強調細節仍待協商
烏克蘭同意停火30天 美國重啟情報共享軍援
不吵了？白宮中東特使曝 澤倫斯基寫信致歉川普
不吵了？白宮中東特使曝 澤倫斯基寫信致歉川普


In [8]:
# Limit to k results and get specific fields
news_items = queryset.values('category', 'title', 'link', 'photo_link')[:3]

In [9]:
news_items

<QuerySet [{'category': '國際', 'title': '對普亭很生氣 川普嗆關稅升級鎖定石油', 'link': 'https://news.ttv.com.tw/news/114033100060001', 'photo_link': 'https://cdn.ttv.com.tw/manasystem/FileData/News/d55e5cae-1837-4125-af5a-eef59ae710b0.jpg'}, {'category': '國際', 'title': '雙普通話推進停火 各有盤算變數眾多', 'link': 'https://news.ttv.com.tw/news/11403240000400W', 'photo_link': 'https://cdn.ttv.com.tw/manasystem/FileData/News/64d14a7a-17c7-4f3b-b98a-e094093b67b7.jpg'}, {'category': '國際', 'title': '雙普通話\u3000同意俄烏能源設施停火30天', 'link': 'https://news.ttv.com.tw/news/114031900022001', 'photo_link': 'https://cdn.ttv.com.tw/manasystem/FileData/News/90bec1c9-e4f5-4d81-be11-bc9bfeda1303.jpg'}]>

In [10]:
dates = list(queryset.values_list('date', flat=True))